In [1]:
public class User
{
    public int Id { get; set; }
    public string Firstname { get; set; }
    public string Lastname { get; set; }
    public DateTime Birthday { get; set; }
    public string Email { get; set; }
    public string Gender { get; set; }
    public string IpAddress { get; set; }
    public float Latitude { get; set; }
    public float Longitude { get; set; }
    public string ImageUrl { get; set; }
    public string Country { get; set; }
    public string City { get; set; }
    public string Address { get; set; }
    public int Age { get; set; }
}

In [1]:
#r "nuget:Microsoft.Data.Analysis,0.2.0"

In [1]:
using Microsoft.Data.Analysis;

var csv = DataFrame.LoadCsv("./MOCK_DATA.csv", ',', true);
var usersDto = new List<User>();
var data = new List<object>();
foreach (var row in csv.Rows)
{
    var user = new User
    {
        Id = Convert.ToInt32(row[csv.Columns.IndexOf("id")]),
        Firstname = row[csv.Columns.IndexOf("first_name")].ToString(),
        Lastname = row[csv.Columns.IndexOf("last_name")].ToString(),
        Birthday = DateTime.Parse(row[csv.Columns.IndexOf("birthday")].ToString()),
        Email = row[csv.Columns.IndexOf("email")].ToString(),
        Gender = row[csv.Columns.IndexOf("gender")].ToString(),
        IpAddress = row[csv.Columns.IndexOf("ip_address")].ToString(),
        Latitude = float.Parse(row[csv.Columns.IndexOf("lat")].ToString()),
        Longitude = float.Parse(row[csv.Columns.IndexOf("long")].ToString()),
        ImageUrl = row[csv.Columns.IndexOf("imageUrl")].ToString(),
        Country = row[csv.Columns.IndexOf("country")].ToString(),
        City = row[csv.Columns.IndexOf("city")].ToString(),
        Address = row[csv.Columns.IndexOf("address")].ToString()
    };
    user.Age = DateTime.Today.Year - user.Birthday.Year;
    usersDto.Add(user);
}

foreach (var user in usersDto)
{
  data.Add(new {
        id = user.Id,
        firstname = user.Firstname,
        lastname = user.Lastname,
        birthday = user.Birthday,
        email = user.Email,
        gender = user.Gender,
        ipAddress = user.IpAddress,
        latitude = user.Latitude,
        longitude = user.Longitude,
        imageUrl = user.ImageUrl,
        country = user.Country,
        city = user.City,
        address = user.Address,
        age = user.Age
  });
}
data

In [1]:
#!javascript
notebookScope.plot = function (plotTarget) {
  let loadPlotly = getJsLoader({
    context: "Users",
    paths: {
      plotly: "https://cdn.plot.ly/plotly-latest.min",
    },
  });

  loadPlotly(["plotly"], (Plotly) => {
    if (typeof document.getElementById(plotTarget) !== "undefined") {
      interactive.csharp.getVariable("data").then((data) => {

        function aggregateData(source, label) {
          var distinctCount = [];
          var labels = [];

          var current = null;
          var cnt = 0;
          for (var i = 0; i < source.length; i++) {
            if (source[i] != current) {
              if (cnt > 0) {
                distinctCount.push(cnt);
                labels.push(current + " " + label);
              }
              current = source[i];
              cnt = 1;
            } else {
              cnt++;
            }
          }
          if (cnt > 0) {
            distinctCount.push(cnt);
            labels.push(current + " " + label);
          }
          return [distinctAgesCount, labels];
        }

        var users = {
          type: "scattergeo",
          mode: "markers",
          geo: "geo",
          lat: data.map((d) => d.latitude),
          lon: data.map((d) => d.longitude),
          text: data.map((d) => [d.firstname, d.lastname].join(" ")),
          marker: {
            symbol: "square",
            color: data.map((d) => d.gender === "Female"? "#D433FF" : "1494C8")
          },
          domain: {
            row: 0,
            column: 0,
          }
        };

        var ages = data.map((d) => d.age);
        ages.sort();
        var agesValues = aggregateData(ages, "years");
        var distinctAgesCount = agesValues[0];
        var ageLabels = agesValues[1];

        var usersAge = {
          name: "Users",
          type: "pie",
          values: distinctAgesCount,
          labels: ageLabels,
          domain: {
            row: 0,
            column: 1,
          }
        };

        var countries = data.map((d) => d.country);
        countries.sort();
        var countriesValues = aggregateData(countries, "");
        var distinctcountryCount = countriesValues[0];
        var countryLabels = countriesValues[1];

        var userCountries = {
          labels: countryLabels,
          values: distinctcountryCount,
          type: "pie",
          textposition: "auto",
          hoverinfo: "none",
          marker: {
            color: "rgb(158,202,225)",
            opacity: 0.6,
            line: {
              color: "rgb(8,48,107)",
              width: 1.5,
            }
          },
          domain: {
            row: 0,
            column: 2,
          }
        };

        var layout = {
          title: "Users",
          grid: { columns: 3, rows: 1 },
          geo: {
            scope: "world",
            showland: true,
            showcountries: true,
            bgcolor: "rgb(90,90,90)",
            landcolor: "rgb(250,250,250)",
          },
        };
        if (typeof document.getElementById(plotTarget) !== "undefined") {
          Plotly.newPlot(plotTarget, [users, usersAge, userCountries], layout);
        }
      });
    }
  });
};

In [1]:
#!html
<div id="plotlyChart"></div>
#!js
notebookScope.plot("plotlyChart");
